### Converting High and Low Grade Glioma to PNG

In [ ]:
import os
import zipfile
import nibabel as nib
from PIL import Image
import numpy as np


#Creating new directories to store the converted files in
os.makedirs("D:\\Gaussian_blur_bicubic_interpolation\\" + "HGG_HR\\" + "t1")
os.makedirs("D:\\Gaussian_blur_bicubic_interpolation\\" + "HGG_HR\\" + "seg")
os.makedirs("D:\\Gaussian_blur_bicubic_interpolation\\" + "HGG_LR\\" + "t1")

#Creating new directories to store the converted files in
os.makedirs("D:\\Gaussian_blur_bicubic_interpolation\\" + "LGG_HR\\" + "t1")
os.makedirs("D:\\Gaussian_blur_bicubic_interpolation\\" + "LGG_HR\\" + "seg")
os.makedirs("D:\\Gaussian_blur_bicubic_interpolation\\" + "LGG_LR\\" + "t1")

#Going to loop through both the HGG and LGG folders in the 'D:\BraTS2018' directory
directory_path = 'D:\BraTS2018'
HGG_LGG = ["HGG", "LGG"]

#New directory images will be saved in
new_directory = "D:\\Gaussian_blur_bicubic_interpolation\\"

#Looping through all patients in the HGG and LGG folders and converting the NIFTI files to png
for folder in HGG_LGG:
    patient_path = directory_path + "\\" + folder + "\\"
    #Listing each patient in the HGG/LGG directory
    list_of_patients = os.listdir(patient_path)
    for patient in list_of_patients:
        #Listing the files in each patients directory
        list_of_files = os.listdir(patient_path + patient)
        for file in list_of_files:
        #Only want to extract the t1.nii.gz and seg.nii.gz files
            if "t1.nii.gz" in file or "seg.nii.gz" in file:
                # Load the NIfTI image
                nifti_image = nib.load(patient_path + patient + "\\" + file)
                # Get the NIfTI data as a NumPy array
                nifti_data = nifti_image.get_fdata()
                # Iterating through the 3rd dimension (slices) and save each slice as a PNG
                # 3rd dimension is the Axial Plane
                for i in range(nifti_data.shape[2]):
                    slice_data = nifti_data[:, :, i]
                    #Removing slices that are empty
                    if np.all(slice_data == 0):
                        continue
                    # Rescale the data to fit the PNG range (0 to 255)
                    min_value = np.min(slice_data)
                    max_value = np.max(slice_data)
                    scaled_data = (slice_data - min_value) / (max_value - min_value) * 255
                    # Convert the NumPy array to a Pillow image
                    png_image = Image.fromarray(scaled_data.astype(np.uint8))
                    # Save the PNG image with a unique name for each slice
                    if "t1.nii.gz" in file:
                        png_image.save(f'{new_directory}\\{folder}_HR\\t1\\{file[:-7]}_slice_{i}.png')
                    if "seg.nii.gz" in file:
                        png_image.save(f'{new_directory}\\{folder}_HR\\seg\\{file[:-7]}_slice_{i}.png')

### Removing slices that don't contain glioma

In [ ]:
#Creating a list of all the slices that are in the seg and t1 folder

import os

patient_names = os.listdir(r"D:\BraTS2018\HGG")

seg_files = os.listdir(r"D:\Gaussian_blur_bicubic_interpolation\HGG_HR\seg")

t1_updated_files_HGG = []

for patient in patient_names:
    for item in seg_files:
        if patient in item:
            t1_updated_files_HGG.append(patient + "_t1_slice_" + item[len(patient)+11:])
             
patient_names = os.listdir(r"D:\BraTS2018\LGG")

seg_files = os.listdir(r"D:\Gaussian_blur_bicubic_interpolation\LGG_HR\seg")

t1_updated_files_LGG = []

for patient in patient_names:
    for item in seg_files:
        if patient in item:
            t1_updated_files_LGG.append(patient + "_t1_slice_" + item[len(patient)+11:])       

In [ ]:
#Removing the slices that are not in Seg folder from t1

for filename in os.listdir(r"D:\Gaussian_blur_bicubic_interpolation\HGG_HR\t1"):
        file_path = os.path.join(r"D:\Gaussian_blur_bicubic_interpolation\HGG_HR\t1", filename)
        if os.path.isfile(file_path) and filename not in t1_updated_files_HGG:
            try:
                os.remove(file_path)
                print(f"Deleted: {filename}")
            except Exception as e:
                print(f"Error deleting {filename}: {e}")    
                
                
for filename in os.listdir(r"D:\Gaussian_blur_bicubic_interpolation\LGG_HR\t1"):
        file_path = os.path.join(r"D:\Gaussian_blur_bicubic_interpolation\LGG_HR\t1", filename)
        if os.path.isfile(file_path) and filename not in t1_updated_files_LGG:
            try:
                os.remove(file_path)
                print(f"Deleted: {filename}")
            except Exception as e:
                print(f"Error deleting {filename}: {e}")      

### Creating Low Resolution Images

In [ ]:
#Downsample.py file can be downloaded from the following link
#https://github.com/wqi/img-downsampler

#Make sure the downsample.py file is in your current working directory

!python downsample.py D:\Gaussian_blur_bicubic_interpolation\HGG_HR\t1 D:\Gaussian_blur_bicubic_interpolation\HGG_LR\t1
!python downsample.py D:\Gaussian_blur_bicubic_interpolation\LGG_HR\t1 D:\Gaussian_blur_bicubic_interpolation\LGG_LR\t1